In [ ]:
from pyspark.sql import SQLContext
sqlContect = SQLContext(sc)

In [ ]:
df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv',\
    format='com.databricks.spark.csv',\
    header='true',\
    inferSchema='true')

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [ ]:
df.describe().toPandas().transpose()

In [ ]:
df.describe('rain_accumulation_9am').show()

In [ ]:
len(df.column), df.count()

In [ ]:
df2 = df.na.drop(subset=['air_pressure_9am'])

In [ ]:
df2.count()

In [ ]:
df2.stat.corr("relative_humidity_9am","relative_humidity_3pm")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
val = df.select('air_temp_9am').flatMap(lambda x: x).collect()

In [ ]:
val = [x for x in val if x is not None]

In [ ]:
num_bins = 50
fig, ax = plt.subplots(figsize=(6,6))
ax.hist(val, bins = num_bins)

In [ ]:
val1 = df.select('max_wind_speed_9am').flatMap(lambda x: x).collect()
val2 = df.select('max_wind_direction_9am').flatMap(lambda x: x).collect()

In [ ]:
for i in range(0, len(val1)):
    if val1[i] == 29.84077959999996:
        print(val2[i])

In [ ]:
removeAllDF = df.na.drop()
removeAllDF.describe(['air_temp_9am']).show()

In [ ]:
removeAllDF.count()

In [ ]:
from pyspark.sql.functions import avg
imputeDF = df

In [ ]:
for x in imputeDF.columns:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    imputeDF = imputeDF.na.fill(meanValue, [x])

In [ ]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

In [ ]:
val3 = removeAllDF.select('air_pressure_9am').flatMap(lambda x: x).collect()

In [ ]:
len([x for x in val3 if x >= 911.736 and x <= 914.67])

In [ ]:
from pyspark.sql.functions import min
imputedDF2 = df
for x in imputedDF2.columns:
    minVal = removeAllDF.agg(min(x)).first()[0]
    print(x, minVal)
    imputedDF2 = imputedDF2.na.fill(minVal, [x])
    
val4 = imputedDF2.select('air_temp_9am').flatMap(lambda x: x).collect()
len([x for x in val4 if x <= 42.292])